In [15]:
import zipfile
import os

# Path to the uploaded ZIP file
zip_file_path = 'DL_Dataset.zip'  # Update with the path where you uploaded DL_Dataset.zip

# Directory to extract the contents
extracted_dir = 'DL_Dataset'  # Update with the desired extraction directory path

# Create a directory for extracted contents
os.makedirs(extracted_dir, exist_ok=True)

# Extract the contents of the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Path to the extracted Train1 directory
train_dir = os.path.join(extracted_dir, 'Train1')

# Verify the contents of the Train1 directory
print(f"Contents of Train1 directory:")
for root, dirs, files in os.walk(train_dir):
    print(f"Root: {root}, Directories: {dirs}, Files: {files}")


Contents of Train1 directory:
Root: DL_Dataset\Train1, Directories: ['Abdullah(ABD)', 'Abdur Rehman Durani', 'Abdur Rehman Sajid', 'Adeen Amir', 'Affan Ali Khan', 'Ahmad Ali Abid', 'Ahmad Fareed sukhera', 'Ali Inayat', 'Arsal Sheikh', 'Basim Mehmood', 'Eman Anjum', 'Faizan Haq', 'Farwa Toor', 'Hammad Anwar', 'Hamza Ahmed Zuberi', 'Hamza Wajid', 'Haya Noor', 'Itba Malahat', 'Lailoma Noor', 'Mia Akbar Jaan', 'Mujtaba', 'Omar Khan', 'Raja', 'Rehan Riaz', 'Saadullah', 'Sameer Shehzad', 'Sheharyar Sadiq', 'Sherry', 'Syed Ibrahim Hamza', 'Talha Wajid', 'Tehrim Ahmed', 'Umair', 'Umer Tayyab', 'Zaid Bin Muzammil', 'Zaid Dandia'], Files: []
Root: DL_Dataset\Train1\Abdullah(ABD), Directories: [], Files: ['Screenshot 2024-05-04 192409.png', 'Screenshot 2024-05-04 192429.png', 'Screenshot 2024-05-04 192449.png', 'Screenshot 2024-05-04 192503.png']
Root: DL_Dataset\Train1\Abdur Rehman Durani, Directories: [], Files: ['Screenshot 2024-05-04 195259.png', 'Screenshot 2024-05-04 195306.png']
Root: DL_D

In [3]:
import face_recognition
import os
import cv2

def load_and_encode_samples(folder_path, scale_factor=1.0):
    samples = {}  # Dictionary to store person name to face encoding mapping

    # Iterate over each subfolder (person) in the training directory
    for person_folder in sorted(os.listdir(folder_path)):
        person_folder_path = os.path.join(folder_path, person_folder)

        if os.path.isdir(person_folder_path):
            # Initialize list to store face encodings for this person
            person_encodings = []

            # Iterate over each image file in the person's folder
            for filename in sorted(os.listdir(person_folder_path)):
                image_path = os.path.join(person_folder_path, filename)
                try:
                    # Load the image using OpenCV
                    image = cv2.imread(image_path)
                    
                    # Resize the image if scale_factor is provided
                    if scale_factor != 1.0:
                        image = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor)
                    
                    # Convert to RGB format for face recognition
                    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    
                    # Detect faces in the image
                    face_locations = face_recognition.face_locations(rgb_image)
                    
                    if len(face_locations) > 0:
                        # Encode all faces found in the image
                        face_encodings = face_recognition.face_encodings(rgb_image, face_locations)
                        person_encodings.extend(face_encodings)
                except Exception as e:
                    print(f"Error processing image {filename}: {str(e)}")
            
            # Store the list of face encodings for this person in the dictionary
            if person_encodings:
                samples[person_folder] = person_encodings
            else:
                print(f"No valid face found in images for {person_folder}")

    return samples


# Example usage:
train_dir = 'DL_Dataset/Train1'  # Update with the path to your Train1 directory
sample_encodings = load_and_encode_samples(train_dir, scale_factor=1.5)

# Display the number of loaded samples
print(f"Number of loaded samples: {sum(len(encodings) for encodings in sample_encodings.values())}")


Number of loaded samples: 158


In [5]:
import cv2
from mtcnn import MTCNN
import face_recognition
import os

# Initialize the MTCNN detector
detector = MTCNN()

# Load and preprocess the target image (image1.jpg)
target_image_path = 'image2.jpg'  # Update with the path to your target image
target_image = cv2.imread(target_image_path)
target_image_rgb = cv2.cvtColor(target_image, cv2.COLOR_BGR2RGB)

# Detect faces using MTCNN
result = detector.detect_faces(target_image_rgb)

# Initialize lists to store face locations and encodings
face_locations = []
face_encodings = []

if result:
    # Extract face locations from MTCNN results
    for face_data in result:
        bounding_box = face_data['box']
        face_locations.append((bounding_box[1], bounding_box[0] + bounding_box[2], bounding_box[1] + bounding_box[3], bounding_box[0]))

    # Extract face encodings from the target image
    face_encodings = face_recognition.face_encodings(target_image_rgb, face_locations)

# Load and encode sample faces from the Train1 dataset
def load_and_encode_samples(folder_path):
    samples = {}  # Dictionary to store person name to face encoding mapping

    # Iterate over each subfolder (person) in the training directory
    for person_folder in sorted(os.listdir(folder_path)):
        person_folder_path = os.path.join(folder_path, person_folder)

        if os.path.isdir(person_folder_path):
            # Initialize list to store face encodings for this person
            person_encodings = []

            # Iterate over each image file in the person's folder
            for filename in sorted(os.listdir(person_folder_path)):
                image_path = os.path.join(person_folder_path, filename)
                try:
                    # Load the image using face_recognition (RGB format)
                    image = face_recognition.load_image_file(image_path)
                    
                    # Detect faces in the image
                    face_locations = face_recognition.face_locations(image)
                    
                    if len(face_locations) == 1:
                        # Encode the face (assuming one face per image)
                        face_encoding = face_recognition.face_encodings(image, face_locations)[0]
                        
                        # Append the face encoding to the list
                        person_encodings.append(face_encoding)
                except Exception as e:
                    print(f"Error processing image {filename}: {str(e)}")
            
            # Store the list of face encodings for this person in the dictionary
            if person_encodings:
                samples[person_folder] = person_encodings
            else:
                print(f"No valid face found in images for {person_folder}")

    return samples

# Path to the extracted Train1 directory
train_dir = 'DL_Dataset/Train1'  # Update with the path to your Train1 directory

# Load and encode sample faces from the Train1 dataset
sample_encodings = load_and_encode_samples(train_dir)

# Initialize a dictionary to store recognized names for each face
recognized_names = {}

# Compare each detected face encoding with sample face encodings
for face_encoding, (top, right, bottom, left) in zip(face_encodings, face_locations):
    # Initialize the recognized name as 'Unknown' by default
    recognized_name = 'Unknown'

    # Initialize minimum distance to a large value
    min_distance = float('inf')

    # Compare the face encoding with sample face encodings
    for name, encodings_list in sample_encodings.items():
        for sample_encoding in encodings_list:
            # Calculate the distance between the detected face and the sample face
            distance = face_recognition.face_distance([sample_encoding], face_encoding)[0]

            # Update recognized name if distance is below threshold
            if distance < min_distance:
                min_distance = distance
                recognized_name = name

    # Store recognized name for this face
    recognized_names[(top, right, bottom, left)] = recognized_name

    # Print recognized name along with bounding box coordinates
    print(f"Recognized: {recognized_name}, Bounding Box: (Top: {top}, Right: {right}, Bottom: {bottom}, Left: {left})")

    # Draw rectangle around the face
    cv2.rectangle(target_image, (left, top), (right, bottom), (0, 255, 0), 2)

    # Draw label with name below the face
    label_size, _ = cv2.getTextSize(recognized_name, cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)
    cv2.rectangle(target_image, (left, bottom - label_size[1] - 10), (right, bottom), (0, 255, 0), cv2.FILLED)
    cv2.putText(target_image, recognized_name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

# Display the annotated image with recognized faces
cv2.imshow("Recognized Faces", target_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Recognized: Lailoma Noor, Bounding Box: (Top: 663, Right: 368, Bottom: 705, Left: 335)
Recognized: Ahmad Ali Abid, Bounding Box: (Top: 496, Right: 714, Bottom: 532, Left: 688)
Recognized: Affan Ali Khan, Bounding Box: (Top: 482, Right: 648, Bottom: 506, Left: 628)
Recognized: Itba Malahat, Bounding Box: (Top: 652, Right: 266, Bottom: 694, Left: 236)
Recognized: Abdur Rehman Sajid, Bounding Box: (Top: 424, Right: 919, Bottom: 453, Left: 896)
Recognized: Zaid Da

import cv2
from mtcnn import MTCNN
import face_recognition
import os

# Initialize the MTCNN detector
detector = MTCNN()

# Load and preprocess the target image (image1.jpg)
target_image_path = 'image1.jpg'  # Update with the path to your target image
target_image = cv2.imread(target_image_path)
target_image_rgb = cv2.cvtColor(target_image, cv2.COLOR_BGR2RGB)

# Detect faces using MTCNN
result = detector.detect_faces(target_image_rgb)

# Initialize lists to store face locations and encodings
face_locations = []
face_encodings = []

if result:
    # Extract face locations from MTCNN results
    for face_data in result:
        bounding_box = face_data['box']
        face_locations.append((bounding_box[1], bounding_box[0] + bounding_box[2], bounding_box[1] + bounding_box[3], bounding_box[0]))

    # Extract face encodings from the target image
    face_encodings = face_recognition.face_encodings(target_image_rgb, face_locations)

# Load and encode sample faces from the Train1 dataset
def load_and_encode_samples(folder_path):
    samples = {}  # Dictionary to store person name to face encoding mapping

    # Iterate over each subfolder (person) in the training directory
    for person_folder in sorted(os.listdir(folder_path)):
        person_folder_path = os.path.join(folder_path, person_folder)

        if os.path.isdir(person_folder_path):
            # Find the first valid image file in the person's folder
            found_face = False
            for filename in sorted(os.listdir(person_folder_path)):
                image_path = os.path.join(person_folder_path, filename)
                try:
                    # Load the image using face_recognition (RGB format)
                    image = face_recognition.load_image_file(image_path)
                    
                    # Detect faces in the image
                    face_locations = face_recognition.face_locations(image)
                    
                    if len(face_locations) == 1:
                        # Encode the face (assuming one face per image)
                        face_encoding = face_recognition.face_encodings(image, face_locations)[0]
                        
                        # Store the face encoding in the dictionary
                        samples[person_folder] = face_encoding
                        found_face = True
                        break  # Stop after processing the first valid image
                except Exception as e:
                    print(f"Error processing image {filename}: {str(e)}")
            
            if not found_face:
                print(f"No valid face found in images for {person_folder}")

    return samples

# Path to the extracted Train1 directory
train_dir = 'DL_Dataset/Train1'  # Update with the path to your Train1 directory

# Load and encode sample faces from the Train1 dataset
sample_encodings = load_and_encode_samples(train_dir)

# Initialize a dictionary to store recognized names for each face
recognized_names = {}




# Compare each detected face encoding with sample face encodings
for face_encoding, (top, right, bottom, left) in zip(face_encodings, face_locations):
    # Initialize the recognized name as 'Unknown' by default
    recognized_name = 'Unknown'
    min_distance = 0.6  # Initialize minimum distance threshold

    # Compare the face encoding with sample face encodings
    for name, sample_encoding in sample_encodings.items():
        # Calculate the distance between the detected face and the sample face
        distance = face_recognition.face_distance([sample_encoding], face_encoding)[0]

        if distance < min_distance:
            min_distance = distance
            recognized_name = name

    # Store recognized name for this face if not already recognized
    if (top, right, bottom, left) not in recognized_names.values():
        recognized_names[(top, right, bottom, left)] = recognized_name

    # Draw rectangle around the face
    cv2.rectangle(target_image, (left, top), (right, bottom), (0, 255, 0), 2)

    # Draw label with name below the face
    label_size, _ = cv2.getTextSize(recognized_name, cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)
    cv2.rectangle(target_image, (left, bottom - label_size[1] - 10), (right, bottom), (0, 255, 0), cv2.FILLED)
    cv2.putText(target_image, recognized_name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)
